# **Preprocessing of images**

In [1]:
!pip install -q -U albumentations
!echo "$(pip freeze | grep albumentations) is successfully installed"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 5.4 MB/s eta 0:00:00
albumentations==1.3.0 is successfully installed


In [2]:
import random

import cv2
from matplotlib import pyplot as plt

import albumentations as A

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
input_folder_path = "/content/drive/MyDrive/train_yolo/data/original"
output_folder_path = "/content/drive/MyDrive/train_yolo/data/images/train"

In [4]:
# adjust this pipeline! https://albumentations.ai/docs/examples/tensorflow-example/

transform = A.OneOf([
    A.GaussNoise(p=1),
    A.MotionBlur(p=1),
    A.Blur(blur_limit=2, p=1),
    A.RandomBrightnessContrast(p=1)
], p=1)

In [5]:
# less blurry and works better! 

transform2 = A.Compose([
    A.OneOf([
        A.IAAAdditiveGaussianNoise(),
        A.GaussNoise(),
    ], p=0.2),
    A.OneOf([
        A.MotionBlur(p=.5),
        A.MedianBlur(blur_limit=3, p=0.1),
        A.Blur(blur_limit=3, p=0.1),
    ], p=0.2),
    A.OneOf([
        A.IAASharpen(),
        A.RandomBrightnessContrast(),            
    ], p=0.3),
])

/usr/local/lib/python3.10/dist-packages/albumentations/imgaug/transforms.py:261: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.10/dist-packages/albumentations/imgaug/transforms.py:231: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)


In [6]:
import os

# Loop through the images in the input folder
for image_name in os.listdir(input_folder_path):
    # Load the image
    image_path = os.path.join(input_folder_path, image_name)
    image = cv2.imread(image_path)

    # Apply the augmentation pipeline
    augmented_image = transform(image=image)['image']

    # Save the augmented image in the output folder
    output_image_path = os.path.join(output_folder_path, image_name)
    cv2.imwrite(output_image_path, augmented_image)


# **Train YOLOv8**

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
ROOT_DIR = '/content/gdrive/My Drive/train_yolo'

In [3]:
!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.4/591.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 24.1 MB/s eta 0:00:00


In [4]:
import os

from ultralytics import YOLO


# Load a model
model = YOLO("yolov8n.yaml")  # build a new model from scratch

# Use the model
results = model.train(data=os.path.join(ROOT_DIR, "google_colab_config.yaml"), epochs=5)  # train the model



                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

In [5]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [6]:
!scp -r /content/runs '/content/gdrive/My Drive/train_yolo'

# **Testing the model**

In [7]:
import os

from ultralytics import YOLO

In [8]:
!yolo task=detect mode=predict model= '/content/gdrive/My Drive/train_yolo/runs/detect/train/weights/best.pt' conf=0.35 source= '/content/gdrive/My Drive/train_yolo/sample_images/' save=True

'/content/gdrive/My Drive/train_yolo'

Ultralytics YOLOv8.0.109 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients

image 1/79 /content/gdrive/My Drive/train_yolo/sample_images/007.jpg: 352x640 (no detections), 91.2ms
image 2/79 /content/gdrive/My Drive/train_yolo/sample_images/16589904889_9b088a8475_b.jpg: 480x640 1 barcode, 52.0ms
image 3/79 /content/gdrive/My Drive/train_yolo/sample_images/20230313_162603.jpg: 640x480 4 barcodes, 62.5ms
image 4/79 /content/gdrive/My Drive/train_yolo/sample_images/21323_label.jpg: 640x640 (no detections), 8.0ms
image 5/79 /content/gdrive/My Drive/train_yolo/sample_images/27473448-94f2d1b6-57ff-11e7-8138-4e22bf107279.png: 640x640 19 barcodes, 7.2ms
image 6/79 /content/gdrive/My Drive/train_yolo/sample_images/329719008_578288697601304_386408813491155454_n.jpg.jpg: 640x640 17 barcodes, 7.2ms
image 7/79 /content/gdrive/My Drive/train_yolo/sample_images/336986271_154871317183099_8397940181163595581_n.jpg: 640x640 

'/content/gdrive/My Drive/train_yolo'

In [9]:
!scp -r /content/runs/detect/predict '/content/gdrive/My Drive/train_yolo/output/'

## **Add a folder in output**

In [10]:
import os
import shutil
import datetime

# Set the destination directory
DEST_DIR = "/content/gdrive/My Drive/train_yolo/output/"

# Find the highest numbered folder in the destination directory
folders = os.listdir(DEST_DIR)
nums = [int(f.split('_')[-1]) for f in folders if f.startswith('predict_') and f.split('_')[-1].isdigit()]
if nums:
    num = max(nums) + 1
else:
    num = 1

# Get the current date and time
now = datetime.datetime.now()
date_time_str = now.strftime("%Y-%m-%d_%H:%M:%S")

# Set the name of the next folder with date and time
next_folder = f"predict_{num}_{date_time_str}"

# Copy the 'predict' folder to the destination directory with the new name
shutil.copytree("/content/runs/detect/predict", os.path.join(DEST_DIR, next_folder))


'/content/gdrive/My Drive/train_yolo/output/predict_1_2023-05-26_16:18:58'